In [1]:
import argparse
from astropy.time import Time
from astroquery.jplsbdb import SBDB
import numpy as np
import pandas as pd

TARGETS = [
    # Atira
    "2020 AV2",
    "163693",

    # Aten
    "2010 TK7",
    "3753",

    # Apollo
    "54509",
    "2063",

    # Amor
    "1221",
    "433",
    "3908",

    # IMB
    "434",
    "1876",
    "2001",

    # MBA
    "2",
    "6",
    "6522",
    "202930",

    # Jupiter Trojans
    "911",
    "1143",
    "1172",
    "3317",

    # Centaur
    "5145",
    "5335",
    "49036",

    # Trans-Neptunian Objects
    "15760",
    "15788",
    "15789",

    # ISOs
    "A/2017 U1"
]

def get_sample_orbits(targets: list[str]) -> pd.DataFrame:
    """
    Query JPL Small-Body Database for orbits of targets.

    Parameters
    ----------
    targets : list[str]
        List of target names.

    Returns
    -------
    orbits: `~pandas.DataFrame`
        DataFrame of containing Keplerian and Cometary elements of each target.
    """
    orbit_dfs = []
    for i, target in enumerate(targets):
        result = SBDB.query(target, full_precision=True, phys=True)

        # Extract the epoch at which the elements are defined
        # and convert it to MJD in TT time scale
        tdb_jd = Time(result["orbit"]["epoch"], scale="tdb", format="jd")
        epoch_df = pd.DataFrame({"mjd_tt": tdb_jd.tt.mjd}, index=[i])

        # Extract the orbital elements and their errors
        elements_df = pd.DataFrame(result["orbit"]["elements"], index=[i])

        # Extract the physical parameters and their errors
        if "G" not in result["phys_par"].keys():
            G = 0.15
        else:
            G = result["phys_par"]["G"]
        phys_df = pd.DataFrame({
                "H": result["phys_par"]["H"],
                "G": G,
            }, 
            index=[i]
        )

        # Combine into a single DataFrame and insert orbit ID
        orbit_i_df = epoch_df.join(elements_df).join(phys_df)
        orbit_i_df.insert(0, "orbit_id", result["object"]["des"])
        orbit_i_df.insert(1, "orbit_name", result["object"]["fullname"])

        orbit_dfs.append(orbit_i_df)

    orbits_df = pd.concat(orbit_dfs)
    return orbits_df

orbits_df = get_sample_orbits(TARGETS)


In [2]:
orbits_df.head()

,orbit_id,orbit_name,mjd_tt,e,e_sig,a,a_sig,q,q_sig,i,...,tp,tp_sig,per,per_sig,n,n_sig,ad,ad_sig,H,G
0,594913,594913 'Aylo'chaxnim (2020 AV2),60000.0,0.177052,9.040800e-07,0.555417,5.694400e-08,0.457080,5.130700e-07,15.868354,...,2.459965e+06,0.000210,151.191588,2.325100e-05,2.381085,3.661800e-07,0.653755,6.702600e-08,16.21,0.15
1,163693,163693 Atira (2003 CP20),60000.0,0.322157,6.797800e-08,0.740999,1.421000e-09,0.502281,5.013300e-08,25.618905,...,2.460044e+06,0.000012,232.983442,6.701600e-07,1.545174,4.444600e-09,0.979717,1.878700e-09,16.39,0.15
2,2010 TK7,(2010 TK7),60000.0,0.190343,2.824400e-07,0.998836,5.719300e-09,0.808714,2.802500e-07,20.900835,...,2.459961e+06,0.000049,364.619190,3.131700e-06,0.987331,8.480100e-09,1.188957,6.807900e-09,20.78,0.15
3,3753,3753 Cruithne (1986 TO),60000.0,0.514969,6.203600e-08,0.997701,1.135700e-09,0.483916,6.188900e-08,19.803779,...,2.459924e+06,0.000015,363.997820,6.215000e-07,0.989017,1.688700e-09,1.511485,1.720500e-09,15.50,0.15
4,54509,54509 YORP (2000 PH5),60000.0,0.230028,1.724100e-05,1.005927,2.273800e-07,0.774536,1.716800e-05,1.599234,...,2.460130e+06,0.112100,368.508807,1.249400e-04,0.976910,3.312300e-07,1.237318,2.796800e-07,22.66,0.15


In [6]:
orbits_df.to_csv("/code/tests/data/sample_orbits.csv")

In [3]:
from precovery.orbit import Orbit
from precovery.orbit import EpochTimescale

orbits_keplerian = []
for i in range(len(orbits_df)):

    orbit_keplerian_i = Orbit.keplerian(
        i, 
        orbits_df["a"].values[i],
        orbits_df["e"].values[i],
        orbits_df["i"].values[i],
        orbits_df["om"].values[i],
        orbits_df["w"].values[i],
        orbits_df["ma"].values[i],
        orbits_df["mjd_tt"].values[i],
        EpochTimescale.TT,
        orbits_df["H"].values[i],
        orbits_df["G"].values[i],
    )
    orbits_keplerian.append(orbit_keplerian_i)

In [4]:
# Three observations daily for two weeks
dts = np.linspace(0, 14, 15)
dts = np.concatenate([dts, dts + 1/24/2, dts + 1/24])
dts.sort()

# Create exposure triplets 
exposure_duration = np.hstack([[30., 60., 90.] for i in range(int(len(dts) / 3))])

# Create list of observatory codes
OBSERVATORY_CODES = ["I11", "I41", "F51"]
observatory_codes = [OBSERVATORY_CODES[j] for j in range(3) for i in range(int(len(dts) / 3))]

In [14]:
initial_epoch = Time(orbits_keplerian[0]._epoch, scale="tt", format="mjd")
observation_times = initial_epoch.utc.mjd + dts + exposure_duration / 86400.
exposure_ids = [f"{obs_i}_{i:06d}" for i, obs_i in enumerate(observatory_codes)]

ephemeris_list = []
for obs_i, time_i in zip(observatory_codes, observation_times):
    ephemeris_list.append(orbits_keplerian[0].compute_ephemeris(obs_i, [time_i])[0])
    
ephemeris_dict = {
    "mjd_utc" : [],
    "ra" : [],
    "dec" : [],
    "mag" : [],
}
for i, eph_i in enumerate(ephemeris_list):
    # PYOORB basic ephemeris
    # modified julian date
    # right ascension (deg)
    # declination (deg)
    # dra/dt sky-motion (deg/day, including cos(dec) factor)
    # ddec/dt sky-motion (deg/day)
    # solar phasae angle (deg)
    # solar elongation angle (deg)
    # heliocentric distance (au)
    # geocentric distance (au)
    # predicted apparent V-band magnitude
    # true anomaly (deg)
    ephemeris_dict["mjd_utc"].append(eph_i._raw_data[0])
    ephemeris_dict["ra"].append(eph_i._raw_data[1])
    ephemeris_dict["dec"].append(eph_i._raw_data[2])
    ephemeris_dict["mag"].append(eph_i._raw_data[9])

ephemeris_df = pd.DataFrame(ephemeris_dict)
ephemeris_df.insert(0, "object_id", orbits_df["orbit_name"].values[0])
ephemeris_df.insert(4, "ra_sigma", 0.)
ephemeris_df.insert(4, "dec_sigma", 0.)
ephemeris_df.insert(6, "mag_sigma", 0.)
ephemeris_df.insert(7, "filter", "V")
ephemeris_df.insert(8, "exposure_id", exposure_ids)
ephemeris_df["observatory_code"] = observatory_codes
ephemeris_df["mjd_start_utc"] = initial_epoch.utc.mjd + dts
ephemeris_df["mjd_mid_utc"] = initial_epoch.utc.mjd + dts + exposure_duration / 86400.
ephemeris_df["exposure_duration"] = exposure_duration
ephemeris_df.insert(1, "obs_id", [f"obs_{i:08d}" for i in range(len(ephemeris_df))])


In [15]:
ephemeris_df.head()

,object_id,obs_id,mjd_utc,ra,dec,dec_sigma,ra_sigma,mag_sigma,filter,exposure_id,mag,observatory_code,mjd_start_utc,mjd_mid_utc,exposure_duration
0,2020 AV2,obs_00000000,59999.999546,327.026372,-19.228343,0.0,0.0,0.0,V,I11_000000,16.957897,I11,59999.999199,59999.999546,30.0
1,2020 AV2,obs_00000001,60000.020727,327.055186,-19.216790,0.0,0.0,0.0,V,I11_000001,16.957951,I11,60000.020033,60000.020727,60.0
2,2020 AV2,obs_00000002,60000.041908,327.084015,-19.205236,0.0,0.0,0.0,V,I11_000002,16.958004,I11,60000.040866,60000.041908,90.0
3,2020 AV2,obs_00000003,60000.999546,328.374059,-18.679829,0.0,0.0,0.0,V,I11_000003,16.959883,I11,60000.999199,60000.999546,30.0
4,2020 AV2,obs_00000004,60001.020727,328.402528,-18.668037,0.0,0.0,0.0,V,I11_000004,16.959931,I11,60001.020033,60001.020727,60.0


In [16]:
ephemeris_df.to_hdf("/code/tests/data/observations.h5", mode="w", key="data", format="table")

In [8]:
exposure_ids

['0_000000',
 '1_000001',
 '2_000002',
 '3_000003',
 '4_000004',
 '5_000005',
 '6_000006',
 '7_000007',
 '8_000008',
 '9_000009',
 '10_000010',
 '11_000011',
 '12_000012',
 '13_000013',
 '14_000014',
 '15_000015',
 '16_000016',
 '17_000017',
 '18_000018',
 '19_000019',
 '20_000020',
 '21_000021',
 '22_000022',
 '23_000023',
 '24_000024',
 '25_000025',
 '26_000026',
 '27_000027',
 '28_000028',
 '29_000029',
 '30_000030',
 '31_000031',
 '32_000032',
 '33_000033',
 '34_000034',
 '35_000035',
 '36_000036',
 '37_000037',
 '38_000038',
 '39_000039',
 '40_000040',
 '41_000041',
 '42_000042',
 '43_000043',
 '44_000044',
 '45_000045',
 '46_000046',
 '47_000047',
 '48_000048',
 '49_000049',
 '50_000050',
 '51_000051',
 '52_000052',
 '53_000053',
 '54_000054',
 '55_000055',
 '56_000056',
 '57_000057',
 '58_000058',
 '59_000059',
 '60_000060',
 '61_000061',
 '62_000062',
 '63_000063',
 '64_000064',
 '65_000065',
 '66_000066',
 '67_000067',
 '68_000068',
 '69_000069',
 '70_000070',
 '71_000071',
 '

In [21]:


ephemeris[0].__dict__

{'_raw_data': array([ 5.97999995e+04,  1.43642448e+02,  1.77718760e+01,  1.46831988e+00,
        -7.36226208e-01,  1.09839227e+01,             nan,  4.79224321e-01,
         1.48022519e+00,  1.61596876e+01,  3.13867828e+02]),
 'mjd': 59799.99954649191,
 'ra': 143.64244781849604,
 'dec': 17.77187595686221,
 'ra_velocity': 1.5419003955202175,
 'dec_velocity': -0.7362262084028952}

In [25]:
orbit_keplerian_i.compute_ephemeris("500", orbit_keplerian_i._epoch + dts)

***ERROR***  9 Dec 2022 15:58:09UTC (Orbit / new) Semimajor Axis is negative.
***ERROR***  9 Dec 2022 15:58:09UTC (Orbit / setParameters) Object has not yet been initialized.


AssertionError: There was an issue with the pyoorb ephemeris generation

In [27]:
orbit_keplerian_i._epoch + dts

array([58080.00000001, 58080.02083335, 58080.04166668, 58081.00000001,
       58081.02083335, 58081.04166668, 58082.00000001, 58082.02083335,
       58082.04166668, 58083.00000001, 58083.02083335, 58083.04166668,
       58084.00000001, 58084.02083335, 58084.04166668, 58085.00000001,
       58085.02083335, 58085.04166668, 58086.00000001, 58086.02083335,
       58086.04166668, 58087.00000001, 58087.02083335, 58087.04166668,
       58088.00000001, 58088.02083335, 58088.04166668, 58089.00000001,
       58089.02083335, 58089.04166668, 58090.00000001, 58090.02083335,
       58090.04166668, 58091.00000001, 58091.02083335, 58091.04166668,
       58092.00000001, 58092.02083335, 58092.04166668, 58093.00000001,
       58093.02083335, 58093.04166668, 58094.00000001, 58094.02083335,
       58094.04166668])